## 데이터 출처

[Naver sentiment movie corpus]: https://github.com/e9t/nsmc/

- CNN 모델의 학습을 위해 [Naver sentiment movie corpus] 데이터셋 중 일부를 추출하여 사용하였습니다.

In [1]:
# torchtext.legacy를 사용할 수 있는 torchtext 버전 설치
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 9.4 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [2]:
#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import torch
# torch.nn : 신경망 구현을 위한 데이터 구조, 신경망 레이어, 관련함수들이 구현되어 있는 팩키지
# torch.nn.functional: torch.nn 팩키지의 함수들이 정의되어 있음 (손실함수, 활성화함수, 풀링함수 등) 
# torch. autograd : 미분을 위한 함수들이 정의되어 있음
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

#import torchtext.data as data
#import torchtext.datasets as datasets
#legacy 버전으로 변경

# torchtext : text의 preprocessing 파이프라인 정의, 
# 토크나이징, Vocab 생성, dataset splits, 데이터 로더 등 지원
from torchtext.legacy import data
import torchtext.datasets as datasets

import pickle

<연습> 3개, 4개, 5개 단어로 된 필터(커널)를 각 100개씩 적용하는 CNN 모델을 정의하여 긍부정 분류를 실행하세요.

분류 정확도를 높일 수 있는 방법을 자유롭게 시도해 보세요.

In [29]:
# CNN 모델링
class CNN_Text(nn.Module):
    # 생성자 : 모델의 구조와 동작을 정의
    # 객체가 갖는 속성값을 초기화함. 객체가 생성될 때 자동으로 호출된다.
    def __init__(self, embed_num, class_num):
        super(CNN_Text, self).__init__() # nn.Module 클래스의 변수들을 상속
        # V: 사전의 크기
        # D: embed_dim
        # C: 분류하고자 하는 클래스의 개수
        # Co : 각 커널(필터)의 갯수
        V = embed_num
        D = 100 
        C = class_num
        
        #--- (연습1) Co에 output channel의 갯수를 지정하세요 ---
        # write code here 
        Co = 100

        #--- (연습2) Ks에 커널사이즈(단어 갯수)를 지정하세요 ---
        # write code here 
        Ks = [3,3,4]

        # 사전에 있는 모든 단어 벡터에 random 초기값
        self.embed = nn.Embedding(V, D) 
        # torch.nn.Conv2d (in_channels, out_channels, kernel_size, stride=1)
        # convs1에 컨볼루션 모듈의 리스트가 들어감 (필터(커널) 갯수만큼) 
        # forward에서 순차적으로 접근 가능
        self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, (K, 100)) for K in Ks])
        self.dropout = nn.Dropout(0.2)
        # nn.Linear : Linear 레이어를 위한 클래스
        self.fc1 = nn.Linear(len(Ks)*Co, C)

# foward 함수 : 모델이 학습데이터를 입력받아서 forward 연산을 진행
# model 객체를 데이터와 함께 호출하면 자동으로 실행된다.
    def forward(self, x):
        x = self.embed(x)  # (N, W, D) 미니배치, 문장 최대길이, 단어벡터 차원
        x = x.unsqueeze(1)  # (N x Ci x W x D) Conv2d를 사용하려면 채널정보 추가해야 함
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks) MaxPool1D는 3D 입력만 받음
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks) max pooling 후에 마지막 차원은 1 -> squeeze
        x = torch.cat(x, 1) # torch.cat(tensors, dim=0), dim=1이면 두번째 차원이 늘어나게 concat (첫번째 차원은 N)
        x = self.dropout(x)  # (N, len(Ks)*Co), dropout을 적용
        logit = self.fc1(x)  # fully-connected layer 적용
        return logit

In [30]:
class mydataset(data.Dataset):
    @staticmethod
    def sort_key(ex):
        return len(ex.text)
    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
        fields = [('text', text_field), ('label', label_field)] # text_field는 text로 호칭하고, label_field 필드는 label로 호칭
        if examples is None:
            path = self.dirname if path is None else path
            examples = []
            for i,line in enumerate(open(path,'r',encoding='utf-8')):
                if i==0:
                    continue
                line = line.strip().split('\t')
                txt = line[1].split(' ')               
                                  
                examples += [ data.Example.fromlist( [ txt, line[2]],fields ) ]
        super(mydataset, self).__init__(examples, fields, **kwargs)

In [31]:
# text_field, label_field : 전처리 관련된 field 객체를 각각 생성 
# batch_first : 미니배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부
# fix_length : 하나의 문장 내 max 토큰수 
# sequential : 시퀀스데이터 여부
text_field = data.Field(batch_first = True, fix_length = 20 )
label_field = data.Field(sequential= False, batch_first = True, unk_token = None )

train_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/aivle/data/nsm/small_ratings_train_tok.txt')

test_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/aivle/data/nsm/small_ratings_test_tok.txt')
#print(test_data.fields.items())

# vocab 생성
text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

# Defines an iterator that loads batches of data from a Dataset
train_iter, test_iter = data.Iterator.splits(
                            (train_data, test_data), 
                            batch_sizes=(100, 1))#, device= 'cuda')
len(text_field.vocab)

21893

In [32]:
# CNN모델 객체를 생성 (embed_num, class_num)
cnn = CNN_Text(len(text_field.vocab), 2)

# torch.optim : 신경망 학습을 위한 다양한 파라미터 최적화 알고리즘이 구현되어 있는 팩키지
# Optimizer를 설정
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()


CNN_Text(
  (embed): Embedding(21893, 100)
  (convs1): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=300, out_features=2, bias=True)
)

In [33]:
for epoch in range(20):
    
    totalloss = 0
    for batch in train_iter:
        optimizer.zero_grad() # resets the gradient to 0
        
        txt = batch.text
        label = batch.label
                
        #print(txt.size()) -> torch.Size([100, 20])
        pred = cnn(txt)
                
        #print(pred.size(), label.size()) -> torch.Size([100, 2]) torch.Size([100])
        #print(label)
        loss = F.cross_entropy(pred, label) # F.cross_entropy는 softmax도 포함되어 있음
        totalloss += loss.data
        
        loss.backward() # backward 연산

        #--- (연습3) 파라미터를 업데이트 하는 함수를 호출하세요 ---
        # write code here
        optimizer.step()
        
        
    print(epoch,'epoch')    
    print('loss : {:.3f}'.format(totalloss.numpy()))

torch.save(cnn,'/content/gdrive/My Drive/Colab Notebooks/aivle/model/cnn_model.pt')

0 epoch
loss : 66.443
1 epoch
loss : 49.384
2 epoch
loss : 37.694
3 epoch
loss : 26.350
4 epoch
loss : 18.141
5 epoch
loss : 12.868
6 epoch
loss : 9.509
7 epoch
loss : 7.200
8 epoch
loss : 5.294
9 epoch
loss : 4.269
10 epoch
loss : 3.369
11 epoch
loss : 2.842
12 epoch
loss : 2.845
13 epoch
loss : 2.463
14 epoch
loss : 2.010
15 epoch
loss : 1.923
16 epoch
loss : 1.828
17 epoch
loss : 1.644
18 epoch
loss : 1.440
19 epoch
loss : 1.394


In [34]:
%%time
from sklearn.metrics import classification_report
cnn.eval()
correct = 0
incorrect = 0
y_test = []
prediction = []

for batch in test_iter:
    txt = batch.text
    label = batch.label
    y_test.append(label.data[0])

    pred = cnn(txt)
    _,ans = torch.max(pred,dim=1)
    prediction.append(ans.data[0])
    
    if ans.data[0] == label.data[0]:        
        correct += 1    
    else:
        incorrect += 1
    
print ('correct : ', correct)
print ('incorrect : ', incorrect)
print(classification_report(torch.tensor(y_test), 
                            torch.tensor(prediction), 
                            digits=4, 
                            target_names=['negative', 'positive']))

# Weighted Avg는 클래스의 수치간의 평균 

correct :  82
incorrect :  18
              precision    recall  f1-score   support

    negative     0.9000    0.7200    0.8000        50
    positive     0.7667    0.9200    0.8364        50

    accuracy                         0.8200       100
   macro avg     0.8333    0.8200    0.8182       100
weighted avg     0.8333    0.8200    0.8182       100

CPU times: user 88.1 ms, sys: 6.04 ms, total: 94.1 ms
Wall time: 92.1 ms
